In [311]:
import pandas as pd
import numpy as np
import os
import string
import seqeval

In [356]:
convert_gold=True
goldfolder='multitask4mix_bi' ## folder containing gold file
#goldfolder = 'mlee_processed_sent'
goldfile='testmlevex.csv' 
#goldfile='test.csv'
golddf = pd.read_csv(os.path.join(goldfolder,goldfile))
opfile=''

convert_pred=True
predfolder='results/multitask4mixbidspec2verbose2_prin_ml_1024_50_bs50_seq3_4bs_2DAYS' ## folder containing pred file
#predfile='EV-EX_wikievent_sent_neg_processed.csv'
print([p for p in os.listdir(predfolder) if 'EX' in p])

[]


In [357]:
predfile = 'EV-EX_mlee_processed_sent.csv' ### if seq3 is not in predfile, i.e. greedy search pred, set this to predfile


if('seq3' in predfolder):
    predfile = 'compare_top_3.csv'
preddf = pd.read_csv(os.path.join(predfolder,predfile))
print(preddf.shape,golddf.shape)

(875, 15) (875, 13)


In [358]:
textcol_g = 'question'
labcol_g='src_label'
labcol_p = 'pred'
if('seq3' in predfolder):
    labcol_p='top_1'

In [359]:
if('subtask' in golddf.columns and golddf.subtask.nunique()>1):
    golddf = golddf[golddf.subtask=='EV-EX']
    
    if(len(preddf)>len(golddf)):
        preddf = preddf.iloc[golddf.index]
    
    #golddf = golddf.reset_index(drop=True)
    #preddf = preddf.reset_index(drop=True)

In [360]:
golddf = golddf.reset_index(drop=True)
preddf = preddf.reset_index(drop=True)

In [361]:
preddf.columns

Index(['text', 'gold', 'top_1', 'top_2', 'top_3', 'top_2_combined', 'cor1',
       'cor2', 'cor3', 'option_1', 'cor_option_1', 'option_2', 'cor_option_2',
       'option_3', 'cor_option_3'],
      dtype='object')

In [362]:
golddf['text'][0]

'EV-EX</s>mlee_processed_sent</s>Adenovirus-mediated gene transfer of endostatin in vivo results in high level of transgene expression and inhibition of tumor growth and metastases.'

In [363]:
golddf.shape,preddf.shape

((875, 13), (875, 15))

In [364]:
def get_BIO(texts,labels,verbose=False):
    
    multi=0
    mlt=[]
    gold=[]
    glabs=[]
    
    textcol_g = 'text'
    labcol_g = 'src_label'
    dftemp = pd.DataFrame(data={'text':texts,'src_label':labels})

    for ix,r in dftemp.iterrows():
        
        mult=False
        
        glabs=[]
        texto = r[textcol_g].split('</s>')[-1].translate(str.maketrans('', '', string.punctuation))
        
        
        
        events = [p.strip() for p in str(r[labcol_g]).split('|')]
        
        labs = [p.split('->')[0].strip() for p in events]
        
        if(verbose):
            print(r['src_label'])
            print(texto)
            print('***************')
            
        senlen = len(texto.split())
        #print(texto,senlen)


        if('NONE' in labs or 'nan' in labs):
            #print('NONE')
            glabs = ['O']*senlen
            gold.append(glabs)
            mlt.append(False)
            #print(glabs,len(glabs))
            continue

        events = [str(p) for p in events if '->' in str(p)]
        
        labs = [p.split('->')[0].strip() for p in events]
        clfs = [p.split('->')[1].strip() for p in events]
        dictword={}
        mct=0
        ml=False
        for ind,g in enumerate(labs):
            startpos = texto.find(g)
            if(labs.count(g)>1):
                mct+=1
                
            if(len(g.split())>1):
                ml=True
                if('ml' in predfolder or 'ML' in predfolder):
                    ml=False 
                    continue
            
            for i,word in enumerate(g.split()):
                
                
                
                if(i==0):
                    #print('found')
                    #startpos = texto.find(word,startpos)
                    
                    
                    dictword[word+str(startpos)] = 'B-'+clfs[ind]
                else:
                    
                    mult=True
                    #print('*******')
                    pos = texto.find(word,startpos+1)
                    if(word_clf):
                        dictword[word+str(startpos)] = 'B-'+clfs[ind]
                    else:
                        dictword[word+str(pos)] = 'I-'+clfs[ind]
            multi+=mult
            
            mult=False
        pos=-1
        mlt.append(ml)
        #print(dictword)
        for word in texto.split():
            pos = texto.find(word,pos+1)
            if(word+str(pos) in dictword.keys()):
                glabs.append(dictword[word+str(pos)])
            else:
                glabs.append('O')

        if(verbose):
            print(glabs,len(glabs))
        gold.append(glabs)
        #print('------------------')
        #break

    print('multi-word triggers ',multi)
    print('multi-class triggers ',mct)
    return gold,mlt


In [365]:
def remove_none(gold,pred):
    g=[]
    p=[]
    for l1,l2 in zip(gold,pred):
        ge = [elem for elem in l1 if elem!='O']
        pe = [l2[ix] for ix,elem in enumerate(l1) if elem!='O']
        g.append(ge)
        p.append(pe)
    return g,p

In [366]:
word_clf=True

In [367]:
preddf[labcol_p][36]

'associated->regulation | metastasis->localization | angiogenesis->blood_vessel_development | expression->gene_expression | coexpression->gene_expression | role->regulation'

In [368]:
goldlabels,mlt1 = get_BIO(golddf[textcol_g],golddf[labcol_g])
predlabels,mlt2 = get_BIO(golddf[textcol_g],preddf[labcol_p])
print(len(goldlabels),len(predlabels))


multi-word triggers  0
multi-class triggers  0
multi-word triggers  0
multi-class triggers  0
875 875


In [369]:
'''df1 = pd.DataFrame(data={'pred':preddf[labcol_p],'gold':golddf[labcol_g],'pred_multi':mlt2})
print(df1[df1.pred_multi==True]['pred'].iloc[1])
print(df1[df1.pred_multi==True]['gold'].iloc[1])'''

"df1 = pd.DataFrame(data={'pred':preddf[labcol_p],'gold':golddf[labcol_g],'pred_multi':mlt2})\nprint(df1[df1.pred_multi==True]['pred'].iloc[1])\nprint(df1[df1.pred_multi==True]['gold'].iloc[1])"

In [370]:
goldpos,predpos = remove_none(goldlabels,predlabels)
print(len(goldpos),len(predpos))


predlabels = np.asarray(predlabels)
goldlabels = np.asarray(goldlabels)

if('NONE' in golddf.src_label.unique()):
    indices = golddf[~golddf.src_label.str.contains('NONE')].index

    predpos2 = predlabels[indices]
    goldpos2 = goldlabels[indices]
else:
    predpos2=predlabels
    goldpos2=goldlabels

875 875


<ipython-input-370-d2b6291ac4b6>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predlabels = np.asarray(predlabels)
<ipython-input-370-d2b6291ac4b6>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  goldlabels = np.asarray(goldlabels)


In [371]:
sr = pd.Series(mlt2)


In [372]:
#goldlabels = goldlabels[2:3]
#predlabels = predlabels[2:3]
#goldpos,predpos = remove_none(goldlabels,predlabels)
#print(len(goldpos),len(predpos))

In [373]:
len(goldlabels),len(goldpos2)

(875, 703)

In [374]:
from seqeval.metrics import f1_score,accuracy_score,precision_score,recall_score

evals=['normal','POS tokens','POS sentences']
precs = [precision_score(goldlabels,predlabels),precision_score(goldpos,predpos),precision_score(goldpos2,predpos2)]
recs = [recall_score(goldlabels,predlabels),recall_score(goldpos,predpos),recall_score(goldpos2,predpos2)]
f1s = [f1_score(goldlabels,predlabels),f1_score(goldpos,predpos),f1_score(goldpos2,predpos2)]
accs = [accuracy_score(goldlabels,predlabels),accuracy_score(goldpos,predpos),accuracy_score(goldpos2,predpos2)]

metrics = pd.DataFrame(data={'evaltype':evals,'precision':precs,'recall':recs,'f1-score':f1s,'accuracy':accs})

#print(f1_score(goldlabels,predlabels),accuracy_score(goldlabels,predlabels))
#print(f1_score(goldpos,predpos),accuracy_score(goldpos,predpos))

In [375]:
metrics[metrics.evaltype.isin(['normal','POS sentences'])]

,evaltype,precision,recall,f1-score,accuracy
0,normal,0.759661,0.789763,0.774420,0.961595
2,POS sentences,0.783725,0.789763,0.786732,0.958017


In [376]:
predfolder

'results/multitask4mixbidspec2verbose2_prin_ml_1024_50_bs50_seq3_4bs_2DAYS'

In [377]:
if('maven' not in predfolder):
    odf.columns

NameError: name 'odf' is not defined

In [ ]:
from seqeval.metrics import classification_report
print(classification_report(goldlabels,predlabels,digits=4))

In [ ]:
predfolder

In [ ]:
#list(goldlabels[0]),list(predlabels[0])

In [ ]:
len(golddf[textcol_g][0].split())
#goldlabels[0]
golddf[textcol_g][0]

In [ ]:
gl=[]
pl=[]
st=[]
for ix,p in enumerate(goldlabels):
    #print(p)
    
    gl+=p
    pl+=predlabels[ix]
    #break
#(goldlabels[0])

st = [s for s in gl if s!='O']
print(pd.Series(st).nunique())
print(len(gl),len(pl))

In [ ]:
import sklearn.metrics as skmt

norm=True
if(norm):
    print('incl NONE class')
    print('precison', skmt.precision_score(gl,pl,average='micro'))
    print('recall', skmt.recall_score(gl,pl,average='micro'))
    print('micro', skmt.f1_score(gl,pl,average='micro'))
    print('macro', skmt.f1_score(gl,pl,average='macro'))
    print('weighted', skmt.f1_score(gl,pl,average='weighted'))
    print('-------------------')
    print('not incl NONE class')
    print('precison', skmt.precision_score(gl,pl,average='micro',labels=st))
    print('recall', skmt.recall_score(gl,pl,average='micro',labels=st))
    print('micro', skmt.f1_score(gl,pl,average='micro',labels=st))
    print('macro', skmt.f1_score(gl,pl,average='macro',labels=st))
    print('weighted', skmt.f1_score(gl,pl,average='weighted',labels=st))
else:
    gl = [gl]
    pl = [pl]
    print('incl NONE class')
    print('precison', skmt.precision_score(gl,pl,average='micro'))
    print('recall', skmt.recall_score(gl,pl,average='micro'))
    print('micro', skmt.f1_score(gl,pl,average='micro'))
    print('macro', skmt.f1_score(gl,pl,average='macro'))
    print('weighted', skmt.f1_score(gl,pl,average='weighted'))
    print('-------------------')
    print('not incl NONE class')
    print('precison', skmt.precision_score(gl,pl,average='micro',labels=st))
    print('recall', skmt.recall_score(gl,pl,average='micro',labels=st))
    print('micro', skmt.f1_score(gl,pl,average='micro',labels=st))
    print('macro', skmt.f1_score(gl,pl,average='macro',labels=st))
    print('weighted', skmt.f1_score(gl,pl,average='weighted',labels=st))

In [ ]:
'''for i in range(5):
    print(golddf[labcol_g][i])
    print(preddf[labcol_p][i])
    print(goldlabels[i])
    print(predlabels[i])'''

In [ ]:
#len(golddf[golddf.src_label=='NONE'])/len(golddf)

In [ ]:
#2659/3456